In [ ]:
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar -xvf spark-3.1.1-bin-hadoop2.7.tgz

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

SyntaxError: invalid character in identifier (<ipython-input-2-a312a4981562>, line 2)

In [ ]:
import findspark
findspark.init()
from google.colab import files
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import isnan, when, count, col, lit
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder

In [ ]:
# import necessary libraries
import pandas as pd 
import numpy
import matplotlib.pyplot as plt 
from pyspark.sql import SparkSession
# create sparksession
sc = SparkSession \
    .builder \
    .appName("Pysparkexample") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
files.upload()
!ls

In [ ]:
data= sc.read.csv("data.csv", inferSchema=True, header=True)
data.printSchema()

In [ ]:
data.describe().toPandas().transpose()

In [ ]:
def replace(column, value):
    return when(column != value, column).otherwise(lit(None))
data = data.withColumn("Market Category", replace(col("Market Category"), "N/A"))
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()
data = data.drop("Market Category")
data = data.na.drop()
print((data.count(), len(data.columns)))

In [ ]:
assembler = VectorAssembler(inputCols=["Year",
                                      "Engine HP",
                                      "Engine Cylinders",
                                      "Number of Doors",
                                      "highway MPG",
                                      "city mpg",
                                      "Popularity"], outputCol = "Attributes")

In [ ]:
regressor = RandomForestRegressor(featuresCol = "Attributes", labelCol="MSR")
pipeline = Pipeline(stages=[assembler, regressor])

In [ ]:
pipeline.write().overwrite().save("pipeline")